In [1]:
pip install pyparsing


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import panel as pn
from panel.interact import interact
pn.extension('tabulator')
import numpy as np
import hvplot.pandas 
import holoviews as hv 
from holoviews import opts 
hv.extension('bokeh')

In [3]:
## loading or uploading the dataset into Jupyter notebook

df =pd.read_csv('/Users/yaabenewaa/Documents/Documents - YAA’s MacBook Air/Dataset/Auto Sales data.csv')
print("DataFrame created successfully:", df.head())

df.rename(columns={'ORDERNUMBER':'ORDER_NUMBER',
                   'QUANTITYORDERED':'QUANTITY_ORDERED',
                   'PRICEEACH':'PRICE_EACH',
                   'ORDERLINENUMBER':'ORDER_LINE_NUMBER',
                   'ORDERDATE':'ORDER_DATE',
                   'PRODUCTLINE':'PRODUCT_LINE',
                   'DEALSIZE':'DEAL_SIZE'}, inplace=True)

print("DataFrame after renaming columns:", df.head())

DataFrame created successfully:    ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
0        10107               30      95.70                2  2871.00   
1        10121               34      81.35                5  2765.90   
2        10134               41      94.74                2  3884.34   
3        10145               45      83.26                6  3746.70   
4        10168               36      96.66                1  3479.76   

    ORDERDATE  DAYS_SINCE_LASTORDER   STATUS  PRODUCTLINE  MSRP PRODUCTCODE  \
0  24/02/2018                   828  Shipped  Motorcycles    95    S10_1678   
1  07/05/2018                   757  Shipped  Motorcycles    95    S10_1678   
2  01/07/2018                   703  Shipped  Motorcycles    95    S10_1678   
3  25/08/2018                   649  Shipped  Motorcycles    95    S10_1678   
4  28/10/2018                   586  Shipped  Motorcycles    95    S10_1678   

           CUSTOMERNAME             PHONE                   

In [4]:
## Performing Data cleaning 

df.drop_duplicates(inplace=True)

In [5]:
df.isnull().sum()

ORDER_NUMBER            0
QUANTITY_ORDERED        0
PRICE_EACH              0
ORDER_LINE_NUMBER       0
SALES                   0
ORDER_DATE              0
DAYS_SINCE_LASTORDER    0
STATUS                  0
PRODUCT_LINE            0
MSRP                    0
PRODUCTCODE             0
CUSTOMERNAME            0
PHONE                   0
ADDRESSLINE1            0
CITY                    0
POSTALCODE              0
COUNTRY                 0
CONTACTLASTNAME         0
CONTACTFIRSTNAME        0
DEAL_SIZE               0
dtype: int64

In [6]:
# This code is used to identify columns associated with the dataset

df.columns

Index(['ORDER_NUMBER', 'QUANTITY_ORDERED', 'PRICE_EACH', 'ORDER_LINE_NUMBER',
       'SALES', 'ORDER_DATE', 'DAYS_SINCE_LASTORDER', 'STATUS', 'PRODUCT_LINE',
       'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'CITY',
       'POSTALCODE', 'COUNTRY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEAL_SIZE'],
      dtype='object')

In [7]:
df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'], dayfirst=True)


In [8]:
df.dtypes

ORDER_NUMBER                     int64
QUANTITY_ORDERED                 int64
PRICE_EACH                     float64
ORDER_LINE_NUMBER                int64
SALES                          float64
ORDER_DATE              datetime64[ns]
DAYS_SINCE_LASTORDER             int64
STATUS                          object
PRODUCT_LINE                    object
MSRP                             int64
PRODUCTCODE                     object
CUSTOMERNAME                    object
PHONE                           object
ADDRESSLINE1                    object
CITY                            object
POSTALCODE                      object
COUNTRY                         object
CONTACTLASTNAME                 object
CONTACTFIRSTNAME                object
DEAL_SIZE                       object
dtype: object

In [9]:
df['COUNTRY'].unique()

array(['USA', 'France', 'Norway', 'Australia', 'Finland', 'UK', 'Spain',
       'Sweden', 'Singapore', 'Canada', 'Japan', 'Italy', 'Denmark',
       'Belgium', 'Philippines', 'Germany', 'Switzerland', 'Ireland'],
      dtype=object)

In [10]:
# This syntax is used to create an interactive widget which will be used through out the code 

idf = df.interactive()
Year_Slider = pn.widgets.IntSlider(name='Date_slider', 
                                    start=2018, 
                                    end=2020, 
                                    value=2018)

Year_Slider


IntSlider(end=2020, name='Date_slider', start=2018, value=2018)

In [11]:
Year_varables = pn.widgets.Select(name='SALES ACROSS THE YEARS FILTER', 
                                  options=['SALES', 'MSRP','QUANTITY_ORDERED'])

In [12]:
countries = ['USA', 'France', 'Norway', 'Australia', 'Finland', 'UK', 'Spain',
       'Sweden', 'Singapore', 'Canada', 'Japan', 'Italy', 'Denmark',
       'Belgium', 'Philippines', 'Germany', 'Switzerland', 'Ireland']

Year_varables_pipeline = (
    idf[
        (idf.ORDER_DATE.dt.year <=Year_Slider) &
        (idf.COUNTRY.isin(countries))
    ]
    .groupby(['ORDER_DATE'])[Year_varables].sum()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)

)

In [13]:
Year_varables_pipeline

In [14]:
# Plotting interactive hvplot in line chart

Year_plot = Year_varables_pipeline.hvplot(kind='line', x='ORDER_DATE',
                                y=Year_varables,
                                title='AUTOMOBILE SALES ACROSS THE YEARS 2018 AND 2020 ',
                                width=600,
                                height = 360,
                                
                            )
Year_plot

In [15]:
Product_varables = pn.widgets.Select(name='PRODUCT LINE FILTER', 
                                  options=['SALES', 'DAYS_SINCE_LASTORDER',])

In [16]:
product = ['Motorcycles', 'Classic Cars', 'Trucks and Buses', 
           'Vintage Cars','Planes', 'Ships', 'Trains']

Product_varables_pipeline = (
    idf[
        (idf.ORDER_DATE.dt.year <=Year_Slider) &
        (idf.PRODUCT_LINE.isin(product))
    ]
    .groupby(['PRODUCT_LINE'])[Product_varables].mean().sort_values(ascending=False)
    .to_frame()
    .reset_index()
    .reset_index(drop=True)

)

In [17]:
# Plotting interactive hvplot in bar chart

Product_plot = Product_varables_pipeline.hvplot(kind='bar', x='PRODUCT_LINE',
                                y=Product_varables,
                                title='SALES MADE ACROSS THE YEARS',
                                width=620,
                                height = 460,
                                rot = 90
                                
                            )
                                
Product_plot

In [18]:
df['PRICE_EACH'].unique()

array([95.7 , 81.35, 94.74, ..., 55.69, 97.16, 62.24])

In [19]:
Price_range = list(range(1,250))

Price_varables_pipeline = (
    idf[
        (idf.ORDER_DATE.dt.year <=Year_Slider) &
        (~(idf.PRICE_EACH.isin(Price_range)))
    ]
    .groupby(['PRICE_EACH','SALES','DEAL_SIZE'])
    .size()
    .to_frame(name='count')
    .reset_index()
 
    
)


In [20]:
# Plotting interactive hvplot using  scatter diagram

Scatterplot = Price_varables_pipeline.hvplot(x='PRICE_EACH', 
                                            y='SALES', 
                                            size=4.5, kind="scatter",
                                            by='DEAL_SIZE',
                                            legend='top_left', 
                                            height=360,
                                            title='SCATTER PLOT DISPLAYING THE DEAL SIZE ACROSS YEARS',
                                            width=600)

Scatterplot

In [21]:
df.columns

Index(['ORDER_NUMBER', 'QUANTITY_ORDERED', 'PRICE_EACH', 'ORDER_LINE_NUMBER',
       'SALES', 'ORDER_DATE', 'DAYS_SINCE_LASTORDER', 'STATUS', 'PRODUCT_LINE',
       'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'CITY',
       'POSTALCODE', 'COUNTRY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEAL_SIZE'],
      dtype='object')

In [22]:
bar_varables = pn.widgets.Select(name='COUNTRY SALES FILTER', 
                                  options=['SALES','QUANTITY_ORDERED','DAYS_SINCE_LASTORDER','MSRP'])

In [23]:
countries = ['USA', 'France', 'Norway', 'Australia', 'Finland', 'UK', 'Spain',
       'Sweden', 'Singapore', 'Canada', 'Japan', 'Italy', 'Denmark',
       'Belgium', 'Philippines', 'Germany', 'Switzerland', 'Ireland']

bar_varables_pipeline = (
    idf[
        (idf.ORDER_DATE.dt.year <=Year_Slider) &
        (idf.COUNTRY.isin(countries))
    ]
    .groupby(['COUNTRY'])[bar_varables].sum().sort_values(ascending=False)
    .to_frame()
    .reset_index()
    .reset_index(drop=True)

)

In [24]:
# Plotting of interactive hvplot using bar chart

bar_plot = bar_varables_pipeline.hvplot(kind='bar', x='COUNTRY',
                                y=bar_varables,
                                title='AUTOMOBILE SALES ACROSS COUNTRIES',
                                width=620,
                                height = 460,
                                rot = 90
                                
                            )
bar_plot

In [25]:
df.columns

Index(['ORDER_NUMBER', 'QUANTITY_ORDERED', 'PRICE_EACH', 'ORDER_LINE_NUMBER',
       'SALES', 'ORDER_DATE', 'DAYS_SINCE_LASTORDER', 'STATUS', 'PRODUCT_LINE',
       'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'CITY',
       'POSTALCODE', 'COUNTRY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEAL_SIZE'],
      dtype='object')

In [26]:
df['CITY'].unique()

array(['NYC', 'Reims', 'Paris', 'Pasadena', 'Burlingame', 'Lille',
       'Bergen', 'Melbourne', 'Newark', 'Bridgewater', 'Nantes',
       'Cambridge', 'Helsinki', 'Stavern', 'Allentown', 'Salzburg',
       'Chatswood', 'New Bedford', 'Liverpool', 'Madrid', 'Lule',
       'Singapore', 'South Brisbane', 'Philadelphia', 'Lyon', 'Vancouver',
       'Burbank', 'New Haven', 'Minato-ku', 'Torino', 'Boras',
       'Versailles', 'San Rafael', 'Nashua', 'Brickhaven', 'North Sydney',
       'Montreal', 'Osaka', 'White Plains', 'Kobenhavn', 'London',
       'Toulouse', 'Barcelona', 'San Diego', 'Bruxelles', 'Tsawassen',
       'Boston', 'Cowes', 'Oulu', 'San Jose', 'Graz', 'Makati City',
       'Marseille', 'Koln', 'Gensve', 'Reggio Emilia', 'Frankfurt',
       'Espoo', 'Dublin', 'Manchester', 'Aaarhus', 'Glendale', 'Sevilla',
       'Brisbane', 'Strasbourg', 'Las Vegas', 'Oslo', 'Bergamo',
       'Glen Waverly', 'Munich', 'Charleroi'], dtype=object)

In [27]:
# Calculating the mean os sale and saving it as mean_of_sale

df['MEAN_OF_SALES']=df['SALES'].mean().astype(int)


In [28]:
# Calculating the max os sale and saving it as max_of_sale

df['MAX_OF_SALE']=df['SALES'].max()
df['MAX_OF_SALE']=df['MAX_OF_SALE'].astype(int)

In [29]:
# Calculating the min os sale and saving it as min_of_sale

df['MIN_OF_SALES']=df['SALES'].min()
df['MIN_OF_SALES']=df['MIN_OF_SALES'].astype(int)

In [30]:
status = pn.widgets.Select(name='TABLE FILTER', options=['SALES','MEAN_OF_SALES','MAX_OF_SALE','MIN_OF_SALES'])


In [31]:
cities = ['NYC', 'Reims', 'Paris', 'Pasadena', 'Burlingame', 'Lille',
       'Bergen', 'Melbourne', 'Newark', 'Bridgewater', 'Nantes',
       'Cambridge', 'Helsinki', 'Stavern', 'Allentown', 'Salzburg',
       'Chatswood', 'New Bedford', 'Liverpool', 'Madrid', 'Lule',
       'Singapore', 'South Brisbane', 'Philadelphia', 'Lyon', 'Vancouver',
       'Burbank', 'New Haven', 'Minato-ku', 'Torino', 'Boras',
       'Versailles', 'San Rafael', 'Nashua', 'Brickhaven', 'North Sydney',
       'Montreal', 'Osaka', 'White Plains', 'Kobenhavn', 'London',
       'Toulouse', 'Barcelona', 'San Diego', 'Bruxelles', 'Tsawassen',
       'Boston', 'Cowes', 'Oulu', 'San Jose', 'Graz', 'Makati City',
       'Marseille', 'Koln', 'Gensve', 'Reggio Emilia', 'Frankfurt',
       'Espoo', 'Dublin', 'Manchester', 'Aaarhus', 'Glendale', 'Sevilla',
       'Brisbane', 'Strasbourg', 'Las Vegas', 'Oslo', 'Bergamo',
       'Glen Waverly', 'Munich', 'Charleroi']

city_status_pipeline = (
    idf[
        (idf.ORDER_DATE.dt.year <= Year_Slider) &
        (idf.CITY.isin(cities))   
    ]
    .groupby('CITY')[status].sum().sort_values(ascending=False)
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [32]:
# Plotting interactive hvplot using bar chart

status_plot = city_status_pipeline.hvplot(x='CITY',kind='bar',
                                          size=9,
                                          y=status,
                                          title = 'CITIES AND THIER AUTOMOBILE SALES',
                                          rot=90,
                                          width=620,
                                          height = 460
                                    
                                         )
status_plot

In [33]:
# Plotting interactive hvplot using table

status_table = city_status_pipeline.pipe(pn.widgets.Tabulator,
                                        pagination = 'remote',
                                        page_size=11,
                                        sizing_mode='stretch_width'
                                     
                                        )
status_table

In [34]:
# Finally the interactive dashboard

template = pn.template.FastListTemplate(
    theme='default',
    title='AUTOMOBILE SALES DASHBOARD',
    sidebar=[
        pn.pane.Markdown("## A LIITLE BIT ABOUT THE DATASET USED"),
        pn.pane.Markdown("#### 'Analyzing automobile sales from 2018 to 2020, the dataset explores purchasing patterns across countries and cities. This comprehensive study delves into consumer preferences, market trends, and regional variations, offering a nuanced understanding of the dynamic automotive landscape. By dissecting this data, valuable insights emerge, guiding strategic decisions and optimizing sales approaches. The dataset meticulous analysis informs the industry on factors influencing performance, enabling a proactive response to market dynamics. Beyond raw numbers, it unveils a narrative of consumer behavior, empowering stakeholders to refine strategies and elevate overall performance in the ever-evolving realm of automobile sales."),
        pn.pane.JPG('/Users/yaabenewaa/Documents/Documents - YAA’s MacBook Air/Datasets/images.jpeg', sizing_mode='scale_both', height=200),
        pn.pane.Markdown("## FILTERS"),
        Year_Slider,
        status,Year_varables,Product_varables,bar_varables
    ],
    main=[
        pn.Row(
            pn.Column(status_table),
            pn.Column(Year_plot),
            
           
        ), 
        pn.Row(
            pn.Column(status_plot),
            pn.Column(Scatterplot.panel(width=600), margin=(136,100)),
            
        ),
         pn.Row(
            pn.Column(Product_plot),
            pn.Column(bar_plot)),
    ],
    accent_base_color="#8d5524",
    header_background="#454545",
)

# Display the template
#template.show()
template.servable();
